In [ ]:
import requests
from bs4 import BeautifulSoup as beau
import sqlite3
import pandas as pd

In [ ]:
url = "http://books.toscrape.com"
response = requests.get(url)
print(response.text)

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [ ]:
soup = beau(response.text , 'html.parser')
connection = sqlite3.connect("Books.db")
cursor = connection.cursor()

createTable = "CREATE TABLE if not exists books(title varchar(255), price int, rating varchar(255) , numrate int ,category_name varchar(255), category_id int , categoryitems int )"
cursor.execute(createTable)
createTable = "CREATE TABLE if not exists category(name varchar(255), id int)"
cursor.execute(createTable)
for i in range(3 , 52):
    print(f'step number {i}')
    cate_url = url +"/"+ soup.find_all("li")[i].find('a').attrs['href']
    response_cate = requests.get(cate_url)
    soup_cate = beau(response_cate.text , 'html.parser')
    cate_name= soup_cate.find_all("strong")[0].get_text()
    sql = "INSERT INTO category (name,id) VALUES (?,?)"
    data = [cate_name,i]
    cursor.execute(sql, data)
    print(f'Category name : {cate_name}')
    items_number = int(soup_cate.find_all("strong")[1].get_text())
    print(f'Books count in this Category : {items_number}')
    if items_number > 20:
        items_number = 20
    x =soup_cate.find_all('article', attrs={'class':'product_pod'})
    for ele in range(items_number):
        
        title = x[ele].find('h3').find('a').attrs['title']
        price = x[ele].find('p', attrs={'class':'price_color'}).get_text()[2:]
        rating = x[ele].find('p').attrs['class'][1]
        if rating == "One":
            numrate = 1
        elif rating == "Two":
            numrate = 2
        elif rating == "Three":
            numrate = 3
        elif rating == "Four":
            numrate = 4
        elif rating == "Five":
            numrate = 5
        sql = "INSERT INTO books (title,price,rating,numrate, category_name ,category_id  ,categoryitems) VALUES (?,?,?,?,?,?,?)"
        data = [title,price,rating ,numrate ,cate_name,i,items_number]
        cursor.execute(sql, data)
        connection.commit()
    print(f'\n************************')
    
        
connection.close()

step number 3
Category name : Travel
Books count in this Category : 11

************************
step number 4
Category name : Mystery
Books count in this Category : 32

************************
step number 5
Category name : Historical Fiction
Books count in this Category : 26

************************
step number 6
Category name : Sequential Art
Books count in this Category : 75

************************
step number 7
Category name : Classics
Books count in this Category : 19

************************
step number 8
Category name : Philosophy
Books count in this Category : 11

************************
step number 9
Category name : Romance
Books count in this Category : 35

************************
step number 10
Category name : Womens Fiction
Books count in this Category : 17

************************
step number 11
Category name : Fiction
Books count in this Category : 65

************************
step number 12
Category name : Childrens
Books count in this Category : 29

************

# 01

In [ ]:
connection = sqlite3.connect("Books.db")
df = pd.read_sql("SELECT * FROM books WHERE numrate >= 3", connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Four,4,Travel,3,11
1,See America: A Celebration of Our National Par...,48.87,Three,3,Travel,3,11
2,Under the Tuscan Sun,37.33,Three,3,Travel,3,11
3,A Year in Provence (Provence #1),56.88,Four,4,Travel,3,11
4,Neither Here nor There: Travels in Europe,38.95,Three,3,Travel,3,11
...,...,...,...,...,...,...,...
285,The Bulletproof Diet: Lose up to a Pound a Day...,49.05,Three,3,Health,48,4
286,10-Day Green Smoothie Cleanse: Lose Up to 15 P...,49.71,Five,5,Health,48,4
287,The Art and Science of Low Carbohydrate Living,52.98,Five,5,Health,48,4
288,Why the Right Went Wrong: Conservatism--From G...,52.65,Four,4,Politics,49,3


# 02

In [ ]:
df = pd.read_sql("SELECT * FROM books WHERE numrate >= 3 OR price > 20 ", connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,It's Only the Himalayas,45.17,Two,2,Travel,3,11
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Four,4,Travel,3,11
2,See America: A Celebration of Our National Par...,48.87,Three,3,Travel,3,11
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,Two,2,Travel,3,11
4,Under the Tuscan Sun,37.33,Three,3,Travel,3,11
...,...,...,...,...,...,...,...
470,Libertarianism for Beginners,51.33,Two,2,Politics,49,3
471,Why the Right Went Wrong: Conservatism--From G...,52.65,Four,4,Politics,49,3
472,Equal Is Unfair: America's Misguided Fight Aga...,56.86,One,1,Politics,49,3
473,Amid the Chaos,36.58,One,1,Cultural,50,1


# 03

In [ ]:
df = pd.read_sql("SELECT * FROM books WHERE rating !='Three'", connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,It's Only the Himalayas,45.17,Two,2,Travel,3,11
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Four,4,Travel,3,11
2,Vagabonding: An Uncommon Guide to the Art of L...,36.94,Two,2,Travel,3,11
3,A Summer In Europe,44.34,Two,2,Travel,3,11
4,The Great Railway Bazaar,30.54,One,1,Travel,3,11
...,...,...,...,...,...,...,...
413,Libertarianism for Beginners,51.33,Two,2,Politics,49,3
414,Why the Right Went Wrong: Conservatism--From G...,52.65,Four,4,Politics,49,3
415,Equal Is Unfair: America's Misguided Fight Aga...,56.86,One,1,Politics,49,3
416,Amid the Chaos,36.58,One,1,Cultural,50,1


# 04

In [ ]:
df = pd.read_sql("SELECT * FROM books WHERE rating IN ('Four' , 'One' , 'Three') AND price BETWEEN 10 AND 40 ", connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,Under the Tuscan Sun,37.33,Three,3,Travel,3,11
1,The Great Railway Bazaar,30.54,One,1,Travel,3,11
2,The Road to Little Dribbling: Adventures of an...,23.21,One,1,Travel,3,11
3,Neither Here nor There: Travels in Europe,38.95,Three,3,Travel,3,11
4,"In a Dark, Dark Wood",19.63,One,1,Mystery,4,20
...,...,...,...,...,...,...,...
176,You Are a Badass: How to Stop Doubting Your Gr...,12.08,Three,3,Self Help,42,5
177,The Girl You Left Behind (The Girl You Left Be...,15.79,One,1,Historical,43,2
178,Blue Like Jazz: Nonreligious Thoughts on Chris...,25.77,One,1,Christian,44,3
179,The Grownup,35.88,One,1,Short Stories,46,1


# 05

In [ ]:
df = pd.read_sql("SELECT * FROM books ORDER BY price DESC LIMIT 5 ", connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,The Perfect Play (Play by Play #1),59.99,Three,3,Romance,9,20
1,Civilization and Its Discontents,59.95,Two,2,Psychology,27,7
2,The Bone Hunters (Lexy Vaughan & Steven Macaul...,59.71,Three,3,Thriller,38,11
3,Thomas Jefferson and the Tripoli Pirates: The ...,59.64,One,1,History,33,18
4,Boar Island (Anna Pigeon #19),59.48,Three,3,Mystery,4,20


# 06

In [ ]:
df = pd.read_sql("SELECT * FROM books ORDER BY numrate DESC , price ASC LIMIT 3,10" , connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,Walt Disney's Alice in Wonderland,12.96,Five,5,Childrens,12,20
1,Princess Between Worlds (Wide-Awake Princess #5),13.34,Five,5,Fantasy,20,20
2,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,Five,5,Sequential Art,6,20
3,Some Women,13.73,Five,5,Womens Fiction,10,17
4,The Epidemic (The Program 0.6),14.44,Five,5,Young Adult,22,20
5,A Gentleman's Position (Society of Gentlemen #3),14.75,Five,5,Romance,9,20
6,"Hyperbole and a Half: Unfortunate Situations, ...",14.75,Five,5,Humor,31,10
7,Fifty Shades Freed (Fifty Shades #3),15.36,Five,5,Adult Fiction,30,1
8,The Collected Poems of W.B. Yeats (The Collect...,15.42,Five,5,Poetry,24,19
9,Sophie's World,15.94,Five,5,Philosophy,8,11


# 07

In [ ]:
cursor = connection.cursor()
for item in range(5):
    
    sql = "INSERT INTO books (title,price,rating,numrate, category_name ,category_id  ,categoryitems) VALUES (?,?,?,?,?,?,?)"
    data = ["book {item}" ,item *10,"one" ,1 ,"Epsilon",52,5]
    cursor.execute(sql, data)
    connection.commit()
print("Done")

Done


# 08

In [ ]:
sql = "Update books SET title = 'Ahmed Book' , price = 20 WHERE numrate = 3 AND price <20"
cursor.execute(sql)
connection.commit()
df = pd.read_sql("SELECT * FROM books WHERE numrate = 3 AND price = 20" , connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,Ahmed Book,20,Three,3,Mystery,4,20
1,Ahmed Book,20,Three,3,Historical Fiction,5,20
2,Ahmed Book,20,Three,3,Sequential Art,6,20
3,Ahmed Book,20,Three,3,Classics,7,19
4,Ahmed Book,20,Three,3,Romance,9,20
5,Ahmed Book,20,Three,3,Womens Fiction,10,17
6,Ahmed Book,20,Three,3,Childrens,12,20
7,Ahmed Book,20,Three,3,Nonfiction,14,20
8,Ahmed Book,20,Three,3,Music,15,13
9,Ahmed Book,20,Three,3,Default,16,20


# 09

In [ ]:
sql = "DELETE FROM books WHERE price > 50 AND numrate <= 2"
cursor.execute(sql)
connection.commit()
df = pd.read_sql("SELECT * FROM books WHERE numrate <= 2 AND price > 50" , connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems


# 10

In [ ]:
df = pd.read_sql("SELECT Count(*) FROM books WHERE title LIKE '%secret%' AND price BETWEEN 10 AND 20" , connection)
df

,Count(*)
0,2


# 11

In [ ]:
pdmix = pd.read_sql("SELECT MAX(price) FROM books WHERE numrate = 5" , connection)
dfmin= pd.read_sql("SELECT MIN(price) FROM books WHERE numrate = 5" , connection)
df ,dfmin

(   Count(*)
 0         2,
    MIN(price)
 0       11.38)

# 12

In [ ]:
df = pd.read_sql("SELECT AVG(price) FROM books WHERE numrate = 5" , connection)
df

,AVG(price)
0,35.789231


# 13

In [ ]:
df = pd.read_sql("SELECT SUM(price) FROM books WHERE numrate = 2 AND price BETWEEN 10 AND 40" , connection)
df

,SUM(price)
0,1641.07


# 14

In [ ]:
df = pd.read_sql("SELECT books.category_name , category.name , books.rating FROM books INNER JOIN category ON books.category_name = category.name" , connection)
df

,category_name,name,rating
0,Travel,Travel,Two
1,Travel,Travel,Four
2,Travel,Travel,Three
3,Travel,Travel,Two
4,Travel,Travel,Three
...,...,...,...
469,Health,Health,Five
470,Health,Health,Five
471,Politics,Politics,Four
472,Cultural,Cultural,One


# 15

In [ ]:
df = pd.read_sql("SELECT COUNT(rating) FROM books WHERE price BETWEEN 20 AND 30" , connection)
df

,COUNT(rating)
0,128


# 16

In [ ]:
df = pd.read_sql("SELECT COUNT(category_name) FROM books WHERE categoryitems > 10" , connection)
df

,COUNT(category_name)
0,383


# 17

In [ ]:
df = pd.read_sql("SELECT * FROM books WHERE category_name = (SELECT category_name from books WHERE category_name = 'Music')" , connection)
df

,title,price,rating,numrate,category_name,category_id,categoryitems
0,Rip it Up and Start Again,35.02,Five,5,Music,15,13
1,Our Band Could Be Your Life: Scenes from the A...,57.25,Three,3,Music,15,13
2,How Music Works,37.32,Two,2,Music,15,13
3,Love Is a Mix Tape (Music #1),18.03,One,1,Music,15,13
4,Please Kill Me: The Uncensored Oral History of...,31.19,Four,4,Music,15,13
5,Kill 'Em and Leave: Searching for James Brown ...,45.05,Five,5,Music,15,13
6,This Is Your Brain on Music: The Science of a ...,38.40,One,1,Music,15,13
7,Ahmed Book,20.00,Three,3,Music,15,13
8,No One Here Gets Out Alive,20.02,Five,5,Music,15,13
9,Life,31.58,Five,5,Music,15,13
